In [1]:
import pandas as pd
import numpy as np

# https://github.com/cgcai/SVY21/tree/master/Python
import SVY21

from geopy.distance import vincenty

# Use SVY21 instead
# from pyproj import Proj, transform

In [2]:
# HDB Carpark information 
# https://data.gov.sg/dataset/hdb-carpark-information 
# Coordinate system is based SVY21 System

df = pd.read_csv("hdb-carpark-information.csv")
df['station'] = ""
df['lon'] = ""
df['lat'] = ""
df.head()


,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,park_and_ride_scheme,station,lon,lat
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,NO,,,
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,,,
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,,,
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,NO,,,
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,NO,,,


In [3]:
# Initialize SVY21 to convert SVY21 coordinates to WGS84 latitude and longitude
cv = SVY21.SVY21()

for index, row in df.iterrows():
    lat, lon = cv.computeLatLon(row['y_coord'], row['x_coord'])
#   print lat, lon
    df.loc[index, 'lat'] = lat
    df.loc[index, 'lon'] = lon


In [4]:
df.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,park_and_ride_scheme,station,lon,lat
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,NO,,103.854,1.30106
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,,103.885,1.321
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,,103.845,1.32828
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,NO,,103.835,1.36909
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,NO,,103.847,1.36612


In [5]:
ws = pd.read_csv("weather_stations.csv", names = ['location','latitude','longitude','none'])
ws = ws.drop('none', axis=1)
ws.head()

,location,latitude,longitude
0,Paya Lebar,1.3524,103.9007
1,Macritchie Reservoir,1.3417,103.8338
2,Lower Peirce Reservoir,1.3701,103.8271
3,Tengah,1.3858,103.7114
4,Changi,1.3678,103.9826


In [6]:
for index, row in df.iterrows():
    closest = ""
    closestdist = 100
    case = (row['lat'], row['lon'])
    
    for index2, row2 in ws.iterrows():
        station = (row2['latitude'], row2['longitude'])
        dist = vincenty(case, station).km
#       print case, station, dist
        
        if dist < closestdist:
            closestdist = dist
            closest = row2['location']
    
    df.loc[index, 'station'] = closest
    print row['address'], closest

BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK Dhoby Ghaut 
BLK 98A ALJUNIED CRESCENT Tanjong Katong 
BLK 101 JALAN DUSUN Whampoa 
BLOCK 253 ANG MO KIO STREET 21 Lower Peirce Reservoir 
BLK 302/348 ANG MO KIO ST 31 Ang Mo Kio 
BLOCK 513 ANG MO KIO ST 53 Ang Mo Kio 
BLOCK 728 ANG MO KIO AVE 6 Ang Mo Kio 
BLK 5022 TO 5095 ANG MO KIO INDUSTRIAL PARK 2 Ang Mo Kio 
ANG MO KIO AVENUE 9 Ang Mo Kio 
BLK 256A ANG MO KIO STREET 21 Lower Peirce Reservoir 
BLK 308C ANG MO KIO AVE 1 Ang Mo Kio 
BLK 309B ANG MO KIO ST 31 Ang Mo Kio 
BLK 316B ANG MO KIO STREET 31 Ang Mo Kio 
BLK 255A ANG MO KIO ST 21 Lower Peirce Reservoir 
BLK 588 ANG MO KIO STREET 52 Ang Mo Kio 
BLK 700 ANG MO KIO AVE 6 Ang Mo Kio 
BLK 596 ANG MO KIO STREEET 52 Ang Mo Kio 
BLK 130A ANG MO KIO ST 12 Ang Mo Kio 
BLK 132A ANG MO KIO ST 12 Ang Mo Kio 
BLK 352A ANG MO KIO STREET 32 Ang Mo Kio 
BLK 2A DOVER ROAD Kent Ridge 
BLK 28 DOVER CRESCENT Kent Ridge 
BLK 19A DOVER CRESCENT Kent Ridge 
BLK 29-39 DOVER ROAD Kent Ridge 
BLK 12A DOVER CL

In [7]:
df.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,park_and_ride_scheme,station,lon,lat
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,NO,Dhoby Ghaut,103.854,1.30106
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,Tanjong Katong,103.885,1.321
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,NO,Whampoa,103.845,1.32828
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,NO,Lower Peirce Reservoir,103.835,1.36909
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,NO,Ang Mo Kio,103.847,1.36612


In [8]:
df.groupby('station').size()

station
Admiralty                   116
Admiralty (West)             31
Ang Mo Kio                   87
Boon Lay (East)              19
Boon Lay (West)              19
Botanic Garden                6
Buangkok                    136
Bukit Panjang                96
Bukit Timah                   4
Chai Chee                    92
Changi                        4
Choa Chu Kang (Central)      73
Clementi                     15
Dhoby Ghaut                  17
East Coast Parkway            2
Jurong (East)                34
Jurong (North)               76
Jurong (West)                20
Kampong Bahru                71
Kent Ridge                   36
Khatib                       14
Lower Peirce Reservoir       28
Macritchie Reservoir         11
Marine Parade                29
Newton                        1
Nicoll Highway               26
Pasir Ris (West)             15
Pasir Ris Central            84
Paya Lebar                   32
Punggol                      96
Queenstown                  105


In [10]:
df.groupby('station').size().to_csv("location.csv", encoding='utf-8')